<a href="https://colab.research.google.com/github/rajasafi/Bayesian-DeepLearnin-BDL-/blob/main/Wine_Quality_Bayesian_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install tensorflow-probability

In [ ]:
pip install tensorflow-datasets

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# The dataset
- We use the Wine Quality dataset, which is available in the TensorFlow Datasets.
- We use the red wine subset, which contains 4,898 examples.
- The dataset has 11numerical physicochemical features of the wine,
- the task is to predict the wine quality, which is a score between 0 and 10.
- In this example, we treat this as a regression task.

# Create training and evaluation datasets
- we load the wine_quality dataset using tfds.load(),
- we convert the target feature to float.
- we shuffle the dataset and split it into training and test sets.
- We take the first train_size examples as the train split, and the rest as the test split.

In [ ]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [ ]:
hidden_units = [8, 8]
learning_rate = 0.001

In [ ]:
def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [ ]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

## Experiment 1: standard neural network
We create a standard deterministic neural network model as a baseline.

In [ ]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/wine_quality/white/1.0.0.incompleteMOMF0G/wine_quality-train.tfrecord*...:…

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_dataset

<_BatchDataset element_spec=({'alcohol': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'chlorides': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'citric acid': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'density': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'fixed acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'free sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'pH': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'residual sugar': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'sulphates': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'total sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'volatile acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [ ]:
test_dataset

<_BatchDataset element_spec=({'alcohol': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'chlorides': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'citric acid': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'density': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'fixed acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'free sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'pH': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'residual sugar': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'sulphates': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'total sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'volatile acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [ ]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 59ms/step - loss: 28.3094 - root_mean_squared_error: 5.3207 - val_loss: 26.0830 - val_root_mean_squared_error: 5.1072
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 26.6507 - root_mean_squared_error: 5.1624 - val_loss: 24.7256 - val_root_mean_squared_error: 4.9725
Epoch 3/100
17/17 [==============================] - 0s 11ms/step - loss: 25.2254 - root_mean_squared_error: 5.0225 - val_loss: 23.3917 - val_root_mean_squared_error: 4.8365
Epoch 4/100
17/17 [==============================] - 0s 13ms/step - loss: 23.8419 - root_mean_squared_error: 4.8828 - val_loss: 22.0536 - val_root_mean_squared_error: 4.6961
Epoch 5/100
17/17 [==============================] - 0s 11ms/step - loss: 22.4810 - root_mean_squared_error: 4.7414 - val_loss: 20.7472 - val_root_mean_squared_error: 4.5549
Epoch 6/100
17/17 [==============================] - 0s 15ms/step - loss: 21.1468 - root_mean_squared_e

In [ ]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[0]

In [ ]:
predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.9 - Actual: 5.0
Predicted: 6.1 - Actual: 7.0
Predicted: 6.0 - Actual: 6.0
Predicted: 5.4 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.4 - Actual: 5.0
Predicted: 5.2 - Actual: 6.0
Predicted: 5.1 - Actual: 5.0
Predicted: 5.8 - Actual: 5.0
Predicted: 4.6 - Actual: 4.0


# Bayesian Neural Network
In Bayesian neural networks, a prior distribution is used to represent our beliefs about the weights of the network before we have seen any data. The prior distribution is updated using Bayes’ rule to obtain the posterior distribution, which represents our beliefs about the weights after taking into account the data. The choice of prior can have a significant impact on the performance of the network,

In [ ]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model

In [ ]:
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [ ]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 4s 254ms/step - loss: 27.0064 - root_mean_squared_error: 5.1955 - val_loss: 22.6744 - val_root_mean_squared_error: 4.7602
Epoch 2/500
5/5 [==============================] - 0s 14ms/step - loss: 24.8333 - root_mean_squared_error: 4.9816 - val_loss: 21.7423 - val_root_mean_squared_error: 4.6614
Epoch 3/500
5/5 [==============================] - 0s 14ms/step - loss: 23.8812 - root_mean_squared_error: 4.8853 - val_loss: 23.8325 - val_root_mean_squared_error: 4.8804
Epoch 4/500
5/5 [==============================] - 0s 14ms/step - loss: 22.2847 - root_mean_squared_error: 4.7191 - val_loss: 22.5932 - val_root_mean_squared_error: 4.7517
Epoch 5/500
5/5 [==============================] - 0s 16ms/step - loss: 19.5190 - root_mean_squared_error: 4.4163 - val_loss: 23.9256 - val_root_mean_squared_error: 4.8902
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 22.3543 - root_mean_squared_error: 4.72

In [ ]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.97, min: 5.25, max: 6.33, range: 1.09 - Actual: 5.0
Predictions mean: 6.17, min: 5.49, max: 6.51, range: 1.02 - Actual: 7.0
Predictions mean: 6.05, min: 5.28, max: 6.39, range: 1.11 - Actual: 6.0
Predictions mean: 5.51, min: 4.67, max: 6.13, range: 1.47 - Actual: 5.0
Predictions mean: 6.11, min: 5.5, max: 6.45, range: 0.95 - Actual: 6.0
Predictions mean: 5.3, min: 4.46, max: 5.88, range: 1.42 - Actual: 5.0
Predictions mean: 5.14, min: 4.5, max: 5.75, range: 1.24 - Actual: 6.0
Predictions mean: 5.15, min: 4.49, max: 5.86, range: 1.37 - Actual: 5.0
Predictions mean: 5.89, min: 5.05, max: 6.33, range: 1.27 - Actual: 5.0
Predictions mean: 4.81, min: 4.03, max: 5.83, range: 1.8 - Actual: 4.0


In [ ]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 4s 54ms/step - loss: 36.3023 - root_mean_squared_error: 6.0247 - val_loss: 36.6855 - val_root_mean_squared_error: 6.0565
Epoch 2/500
17/17 [==============================] - 0s 5ms/step - loss: 36.5154 - root_mean_squared_error: 6.0424 - val_loss: 34.1132 - val_root_mean_squared_error: 5.8402
Epoch 3/500
17/17 [==============================] - 0s 6ms/step - loss: 32.7178 - root_mean_squared_error: 5.7196 - val_loss: 34.4571 - val_root_mean_squared_error: 5.8696
Epoch 4/500
17/17 [==============================] - 0s 5ms/step - loss: 31.0550 - root_mean_squared_error: 5.5724 - val_loss: 27.0031 - val_root_mean_squared_error: 5.1961
Epoch 5/500
17/17 [==============================] - 0s 6ms/step - loss: 30.1489 - root_mean_squared_error: 5.4904 - val_loss: 30.3195 - val_root_mean_squared_error: 5.5059
Epoch 6/500
17/17 [==============================] - 0s 7ms/step - loss: 29.9952 - root_mean_squared_error

In [ ]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 4s 91ms/step - loss: 25.3903 - root_mean_squared_error: 5.2515 - val_loss: 23.9144 - val_root_mean_squared_error: 5.1822
Epoch 2/1000
17/17 [==============================] - 0s 10ms/step - loss: 16.8457 - root_mean_squared_error: 5.2202 - val_loss: 22.7032 - val_root_mean_squared_error: 5.1152
Epoch 3/1000
17/17 [==============================] - 0s 10ms/step - loss: 14.6608 - root_mean_squared_error: 5.0312 - val_loss: 14.0094 - val_root_mean_squared_error: 4.8686
Epoch 4/1000
17/17 [==============================] - 0s 10ms/step - loss: 12.4102 - root_mean_squared_error: 4.8608 - val_loss: 16.7875 - val_root_mean_squared_error: 4.9593
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 11.8538 - root_mean_squared_error: 4.9570 - val_loss: 11.1901 - val_root_mean_squared_error: 4.8380
Epoch 6/1000
17/17 [==============================] - 0s 7ms/step - loss: 9.4486 - root_mean_squar

In [ ]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 6.12, stddev: 0.78, 95% CI: [7.64 - 4.6] - Actual: 5.0
Prediction mean: 6.41, stddev: 0.85, 95% CI: [8.07 - 4.74] - Actual: 7.0
Prediction mean: 6.25, stddev: 0.8, 95% CI: [7.82 - 4.69] - Actual: 6.0
Prediction mean: 5.77, stddev: 0.73, 95% CI: [7.21 - 4.33] - Actual: 5.0
Prediction mean: 6.26, stddev: 0.82, 95% CI: [7.86 - 4.66] - Actual: 6.0
Prediction mean: 5.31, stddev: 0.68, 95% CI: [6.63 - 3.99] - Actual: 5.0
Prediction mean: 5.22, stddev: 0.67, 95% CI: [6.53 - 3.91] - Actual: 6.0
Prediction mean: 5.25, stddev: 0.67, 95% CI: [6.57 - 3.94] - Actual: 5.0
Prediction mean: 6.12, stddev: 0.78, 95% CI: [7.64 - 4.6] - Actual: 5.0
Prediction mean: 5.41, stddev: 0.69, 95% CI: [6.76 - 4.06] - Actual: 4.0


# Conclusion :
- The create_baseline_model function defines a simple feedforward neural network with a single hidden layer.
- The create_bnn_model function defines a Bayesian neural network using the DenseVariational layer from TensorFlow Probability. This layer applies a variational inference approach to approximate the posterior distribution of the weights given the data. The make_prior_fn and make_posterior_fn arguments are functions that define the prior and posterior distributions for the weights, respectively. The kl_weight parameter is used to scale the Kullback-Leibler divergence term in the loss function, which is often set to 1/number_of_data_points.
- The create_probablistic_bnn_model function is similar to create_bnn_model, but it uses the IndependentNormal layer from TensorFlow Probability to model the output distribution. This layer creates a multivariate normal distribution with independent components, where the mean and variance are modeled by the output of a Dense layer.

In summary, the create_baseline_model function defines a standard neural network, while the create_bnn_model and create_probablistic_bnn_model functions define Bayesian neural networks that use probabilistic layers and Bayesian inference to account for uncertainty in the model’s predictions. The create_probablistic_bnn_model function further models the output distribution using a multivariate normal distribution with independent components.